<a href="https://colab.research.google.com/github/adasrinivas1229/DEEP-LEARNING-KGP/blob/master/CONVAUTOLOSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import tensorflow as tf
from matplotlib import image
np.random.seed(1)
tf.random.set_seed(1)
np.random.seed(1)
tf.random.set_seed(1)

batch_size = 16         #None    #Set the batch size
epochs = 10             #None    #Set the Number of epochs
learning_rate = 1e-2    #None    #Set the Learning Rate
intermediate_dim = 64   #Intermediate dimension (Please dont change)
original_dim = 784      #Original image size after flattening (Please Dont change)


l = os.listdir('drive/My Drive/ML/datasets/mnist_digits/')   # Load image names from the given diretory hint: use os.listdir

imgs=[] # Initializing imgs list to hold images
for i in l :
  # Read each image using imread and append to imgs (2 lines)
  im= image.imread('drive/My Drive/ML/datasets/mnist_digits/'+i)  # Read image i from specified directory
  imgs.append(im)   # Append the image i into imgs list

training_features=np.array(imgs)                                                # Convert the list into array and store in training_features
training_features = training_features / np.max(training_features)               # Scale the pixel values 1 to 1 scale Hint: divide each pixle with max value 
training_features = training_features.reshape(training_features.shape[0],
                                              training_features.shape[1], training_features.shape[2],1)  # Reshaping the image to add channel and  Hint: use np.reshape
training_features = training_features.astype('float32')                         # Append the read image into imgs list.

training_dataset = tf.data.Dataset.from_tensor_slices(training_features)        #Load data into tensor Hint: use tf.data.Dataset.from_tensor_slices()
training_dataset = training_dataset.batch(batch_size)                           #Partition the data into batches Hint: use Dataset.batch()
training_dataset = training_dataset.shuffle(training_features.shape[0])         #Shuffle the samples 
training_dataset = training_dataset.prefetch(batch_size * 4)  

class Encoder(tf.keras.layers.Layer):
  def __init__(self, intermediate_dim):
    super(Encoder, self).__init__()
    self.hidden_layer1 = tf.keras.layers.Conv2D(filters=intermediate_dim,kernel_size=3,activation=tf.nn.relu, padding='same',kernel_initializer='he_uniform')   # Conv2d layer with relu activation, intermediate_dim filters, kernal size 3, padding same, kernal initializer 'he_uniform'
    self.max_pool1 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding = 'same')       # Maxpooling2d with size 2x2 and padding same.
    self.hidden_layer2 = tf.keras.layers.Conv2D(filters=intermediate_dim//2,kernel_size=3,activation=tf.nn.relu, padding='same',kernel_initializer='he_uniform')   # Conv2d layer with relu activation, intermediate_dim/2 filters, kernal size 3, padding same, kernal initializer 'he_uniform'
    self.max_pool2 = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding = 'same')       # Maxpooling2d with size 2x2 and padding same.
    self.hidden_layer3 = tf.keras.layers.Conv2D(filters=intermediate_dim//4,kernel_size=3,activation=tf.nn.relu, padding='same',kernel_initializer='he_uniform')   # Conv2d layer with relu activation, intermediate_dim/4 filters, kernal size 3, padding same, kernal initializer 'he_uniform'
    self.output_layer = tf.keras.layers.MaxPool2D(pool_size=(2,2), padding = 'same')    # Maxpooling2d with size 2x2 and padding same.
    
  def call(self, input_features):
  # Write the code for call method with calling sequence of layers.
    act1 = self.hidden_layer1(input_features)                              # Define the sequence of execution of layers 
    mp1 = self.max_pool1(act1)
    act2 = self.hidden_layer2(mp1)
    mp2 = self.max_pool2(act2)
    act3 = self.hidden_layer3(mp2)
    output= self.output_layer(act3)
    return output

class Decoder(tf.keras.layers.Layer):
  def __init__(self, intermediate_dim, original_dim):
    super(Decoder, self).__init__()

    self.hidden_layer1 = tf.keras.layers.Conv2D(filters=intermediate_dim//2,kernel_size=3,activation=tf.nn.relu, padding='same',kernel_initializer='he_uniform') #'he_uniform'
    self.upsample1 = tf.keras.layers.UpSampling2D((2,2))
    self.hidden_layer2 = tf.keras.layers.Conv2D(filters=intermediate_dim,kernel_size=3,activation=tf.nn.relu, padding='same',kernel_initializer='he_uniform') #'he_uniform'
    self.upsample2 = tf.keras.layers.UpSampling2D((2,2))
    
   
    self.hidden_layer3 = tf.keras.layers.Conv2D(filters=intermediate_dim,kernel_size=3,activation=tf.nn.relu, padding='same',kernel_initializer='he_uniform') #'he_uniform'
    self.upsample3 = tf.keras.layers.UpSampling2D((2,2))

    self.output_layer = tf.keras.layers.Conv2D(filters=1, kernel_size=5,activation='sigmoid')
   
  def call(self, code):
    act1 = self.hidden_layer1(code)  
    ups1 = self.upsample1(act1)
    act2 = self.hidden_layer2(ups1)  
    ups2 = self.upsample2(act2)
    act3 = self.hidden_layer3(ups2)  
    ups3 = self.upsample3(act3)
    output=self.output_layer(ups3)
    
    return output

#Define Autoencoder class
class Autoencoder(tf.keras.Model):
  def __init__(self, intermediate_dim, original_dim):
    super(Autoencoder, self).__init__()

    #Start your code here...
    self.encoder = Encoder(intermediate_dim=intermediate_dim)                   # initialize encoder in autoencoder object
    self.decoder = Decoder(intermediate_dim=intermediate_dim, original_dim=original_dim)  #initialize decoder in autoencoder object
    #End your code here...

  def call(self, input_features):
    #Start your code here...
    code = self.encoder(input_features)                                         # Define the sequence of execution of encoder and decoder by calling encoder and decoder
    reconstructed = self.decoder(code)
    #End your code here...
    return reconstructed


#Define loss function
def loss(model, original):
  #Start your code here...
  rec_image=model(original)
  reconstruction_error = tf.reduce_mean(tf.square(tf.subtract( tf.reshape(rec_image, [-1]), original)))
  #End your code here...

  return reconstruction_error

# Read the image 'img50.jpg' and use the loss function to calculate the reconstruction error 

im=image.imread('drive/My Drive/ML/datasets/mnist_digits/img50.jpg')
im=im.astype('float32')
d=tf.constant(im)
t=d.shape
d1=tf.reshape(d,(1,t[0],t[1],1))
autoencoder1 = Autoencoder(intermediate_dim=16, original_dim=24)# There may be more statements
l= loss(autoencoder1, d1)
print(l)



tf.Tensor(4813.1978, shape=(), dtype=float32)
